In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import sqlite3

In [3]:
url = 'https://pokemondb.net/pokedex/all'

# Send a GET request to the Pokemon webpage
response = requests.get(url)

In [4]:
print(response.content)

b'<!DOCTYPE html>\n<html lang="en">\n<head>\n<meta charset="utf-8">\n<title>Pok\xc3\xa9mon Pok\xc3\xa9dex: list of Pok\xc3\xa9mon with stats | Pok\xc3\xa9mon Database</title>\n<link rel="preconnect" href="https://img.pokemondb.net">\n<link rel="preconnect" href="https://s.pokemondb.net">\n<link rel="preload" href="/static/fonts/fira-sans-v17-latin-400.woff2" as="font" type="font/woff2" crossorigin>\n<link rel="preload" href="/static/fonts/fira-sans-v17-latin-400i.woff2" as="font" type="font/woff2" crossorigin>\n<link rel="preload" href="/static/fonts/fira-sans-v17-latin-600.woff2" as="font" type="font/woff2" crossorigin>\n<link rel="stylesheet" href="/static/css/pokemondb-aa70195104.css">\n<meta name="viewport" content="width=device-width, initial-scale=1">\n<meta property="og:description" name="description" content="The Pok\xc3\xa9dex contains detailed stats for every creature from the Pok\xc3\xa9mon games, up to and including the latest Scarlet/Violet games.">\n<link rel="canonical" 

In [5]:
# Parse the content of the webpage
soup = BeautifulSoup(response.content, 'html.parser')

# Find the table containing the Pokemon data
table = soup.find('table', {'id': 'pokedex'})

# Extract the table headers
headers = [header.text for header in table.find_all('th')]

# Extract the table rows
rows = table.find_all('tr')[1:]  # Skip the header row

In [6]:
# Extract the data from each row
data = []
for row in rows:
    cells = row.find_all('td')
    cells_text = [cell.text.strip() for cell in cells]
    data.append(cells_text)


df = pd.DataFrame(data, columns=headers)

In [7]:
df.head

<bound method NDFrame.head of          #                     Name           Type Total   HP Attack Defense  \
0     0001                Bulbasaur   Grass Poison   318   45     49      49   
1     0002                  Ivysaur   Grass Poison   405   60     62      63   
2     0003                 Venusaur   Grass Poison   525   80     82      83   
3     0003   Venusaur Mega Venusaur   Grass Poison   625   80    100     123   
4     0004               Charmander           Fire   309   39     52      43   
...    ...                      ...            ...   ...  ...    ...     ...   
1210  1023               Iron Crown  Steel Psychic   590   90     72     100   
1211  1024    Terapagos Normal Form         Normal   450   90     65      85   
1212  1024  Terapagos Terastal Form         Normal   600   95     95     110   
1213  1024   Terapagos Stellar Form         Normal   700  160    105     110   
1214  1025                Pecharunt   Poison Ghost   600   88     88     160   

     Sp. 

In [8]:
# Save the DataFrame to a CSV file
csv_file_path = 'pokemon_data.csv'
df.to_csv(csv_file_path, index=False)

In [9]:
# Save the DataFrame to a database file (SQLite)
# SQLite database file path
db_file_path = 'pokemon_data.db'

# Connect to the SQLite database
conn = sqlite3.connect(db_file_path)

# Save the DataFrame to a SQLite table
df.to_sql('pokemon', conn, if_exists='replace', index=False)

# Close the database connection
conn.close()